## <center> DISNEY'S MOVIES: DATA CLEANING

## TASK 3: CLEANING UP DATA
#### 3.1. Uploading data

In [ ]:
## This is for uploading files on Google Colab
import json

def load_data(title):
  with open(title, encoding = 'utf-8') as f:
    return json.load(f)


from google.colab import files
files.upload()

disney_data = load_data('disney_data.json')

Saving disney_data.json to disney_data.json


There are some issues we should fix through html scraping:
  - The Box office and Released dates have wrong format for datetime and currency. 
  - There are references (eg. [1], [2]) that should be removed. 
  - The datetime is repeated in 2 different formats (June 12, 2010 (2010-06-12))
  - For a key having more than a paired value (eg. Starring), they should be listed.

#### 3.2. Convert Running time into an integer

In [ ]:
## Let's check the Running format first.
[movie.get('Running time', 'N/A') for movie in disney_data]

## There are some movies that do not have the Running time infomation or have a list of Running time. In case there is a list, 
## we will take the first option

In [ ]:
import re

def convert_running_time(running_time):
  '''
  Convert running time by Regex library
  '''
  # If the movie does not have Running time
  if running_time == 'N/A':
    return None
  else:
    # If the Running time is a list
    if isinstance(running_time, list):
      strtime = re.search(r'(?P<strtime>\d{2,3}) (\w*)', running_time[0])
      return int(strtime.group('strtime'))
    else: # if the running time is a string
      strtime = re.search(r'(?P<strtime>\d{2,3}) (\w*)', running_time)
      return int(strtime.group('strtime'))

  
for movie in disney_data:
  movie['Running time'] = convert_running_time(movie.get('Running time', 'N/A'))

print([movie['Running time'] for movie in disney_data])

[41, 83, 88, 126, 74, 64, 70, 42, 70, 71, 75, 94, 73, 75, 82, 68, 74, 96, 75, 84, 77, 92, 69, 81, 60, 127, 93, 76, 75, 73, 85, 81, 70, 90, 80, 75, 83, 83, 72, 97, 75, 104, 93, 24, 105, 95, 97, 134, 24, 69, 92, 126, 79, 97, 128, 73, 91, 105, 98, 130, 89, 93, 67, 98, 100, 118, 103, 110, 80, 74, 91, 91, 97, 118, 139, 131, 92, 87, 116, 93, 110, 110, 131, 101, 108, 84, 78, 75, 164, 106, 110, 99, 113, 108, 102, 85, 91, 93, 100, 100, 79, 96, 113, 89, 117, 92, 88, 92, 87, 93, 93, 93, 90, 83, 96, 88, 89, 91, 93, 92, 97, 100, 100, 89, None, 91, 112, 115, 95, 91, 97, 104, 74, 48, 77, 104, 128, 101, 94, 104, 90, 100, 88, 93, 98, 112, 84, 97, 97, 114, 96, 97, 109, 83, 90, 107, 96, 103, 91, 95, 105, 113, 80, 101, 90, 74, 90, 89, 110, 74, 93, 84, 83, 74, 77, 107, 93, 88, 108, 84, 121, 89, 104, 90, 86, 84, 108, 107, 96, 98, 105, 108, 94, 106, 102, 88, 102, 102, 97, 111, 100, 96, 98, 78, 81, 108, 89, 99, 89, 81, 92, 100, 89, 79, 91, 101, 104, 103, 86, 105, 74, 93, 92, 98, 95, 93, 87, 93, 87, 128, 77, 8

In [ ]:
## Another way to solve this problem

def convert_running_time_2(running_time):
  '''
  Convert running time use string.split. 
  It returns the same output as using Regex.
  '''
  # If the movie does not have Running time
  if running_time == 'N/A':
    return None
  else:
    # If the Running time is a list
    if isinstance(running_time, list):
      strtime = running_time[0].split(' ')[0]
      return int(strtime)
    else: # if the running time is a string
      strtime = running_time.split(' ')[0]
      return int(strtime.group('strtime'))


#### 3.3. Convert Budget into a number

In [ ]:
## Let's check the Budget first

[[movie.get('Budget', 'N/A'), movie['Title']] for movie in disney_data]

The budget current format makes things quite complicated. It includes the different currencies rather than US dollar, \$65 million or \$65,000,000, a range of value, a list and missing values 

So we will try on our best. We will substitute only US currency, other currency without the Dollar equivalent number is omitted. For the range of values and the list of values, we take the first number (eg. \$90-100 million ---> \$90 million )

In [ ]:
## Build function to convert Budget into floats

import re

def get_money_string(string):
  '''
  Return money string from a text
  '''
  
  if 'A$' in string or 'AU$' in string:
    return None

  if '$' not in string:
    return None
  
  else:
    string = string.lower()
    if 'million' in string or 'billion' in string:
      pattern = r'[US]*\$(?P<number>[0-9.]+)[0-9\-\$\s\w\W]* (?P<unit>million|billion)'
      money = re.search(pattern, string)
      return money.group('number') + ' ' + money.group('unit')
    
    else:
      pattern = r'\$([0-9.,]+)'
      money = re.search(pattern, string)
      return money.group(1)


def word_to_number(string):

  if string is not None:

    if 'million' in string:
      string = string.replace(' million', '')
      return float(string) * 1000000

    elif 'billion' in string:
      string = string.replace(' billion', '')
      return float(string) * 1000000000

    else:
      string = string.replace(',', '')
      return float(string)


def get_budget(string):
  '''
  To obtain the money string
  '''
  
  if string in ['N/A', '', 'unknown', 'Unknown']:
    return None
  
  ## If the Budget value is a list
  if isinstance(string, list):
    substring = [s for s in string if '$' in s][0]  # Take the first value
    money_string = get_money_string(substring)
    return word_to_number(money_string)
  
  ## If the Budget value is a string
  else:
    money_string = get_money_string(string)
    return word_to_number(money_string)



In [ ]:
## Let's check the functions

for movie in disney_data:
  try:
    movie['Budget_converted'] = get_budget(movie.get('Budget', 'N/A'))
  except Exception as e:
    print(e)
    print(movie['Title'], movie['Budget'])


'NoneType' object has no attribute 'group'
Ponyo ['¥', '3.4 billion', '(', 'US$', '34 million)']
list index out of range
Jagga Jasoos ['131 crore']


In [ ]:
## Alright, we only have to fix Ponyo and Jagga Jasoos movie

for movie in disney_data:
  if movie['Title'] == 'Ponyo':
    movie['Budget_converted'] = 34000000
  if movie['Title'] == 'Jagga Jasoos':
    movie['Budget_converted'] = None
  
  print([movie.get('Budget', 'N/A'), movie['Budget_converted']])



['N/A', None]
['$1.49 million', 1490000.0]
['$2.6 million', 2600000.0]
['$2.28 million', 2280000.0]
['$600,000', 600000.0]
['$950,000', 950000.0]
['$858,000', 858000.0]
['N/A', None]
['$788,000', 788000.0]
['N/A', None]
['$1.35 million', 1350000.0]
['$2.125 million', 2125000.0]
['N/A', None]
['$1.5 million', 1500000.0]
['$1.5 million', 1500000.0]
['N/A', None]
['$2.2 million', 2200000.0]
['$1,800,000', 1800000.0]
['$3 million', 3000000.0]
['N/A', None]
['$4 million', 4000000.0]
['$2 million', 2000000.0]
['$300,000', 300000.0]
['$1.8 million', 1800000.0]
['N/A', None]
['$5 million', 5000000.0]
['N/A', None]
['$4 million', 4000000.0]
['N/A', None]
['N/A', None]
['N/A', None]
['N/A', None]
['N/A', None]
['N/A', None]
['$700,000', 700000.0]
['N/A', None]
['N/A', None]
['N/A', None]
['N/A', None]
['N/A', None]
['$6 million', 6000000.0]
['under $1 million or $1,250,000', 1000000.0]
['N/A', None]
['N/A', None]
['$2 million', 2000000.0]
['N/A', None]
['N/A', None]
['$2.5 million', 2500000.0]
[

#### 3.4. Convert Box Office into a number

In [ ]:
### Check Box Office format
[[movie['Title'], movie.get('Box office', 'N/A')] for movie in disney_data]

Similar to the Budget, we will try to convert the Box office into an integer. However, ff a movie has more than a value, we will take their sum instead of the first number

In [ ]:
def get_box_office(string):
  '''
  To obtain the Box office string
  '''
  
  if string in ['N/A', '', 'unknown', 'Unknown', 'est.']:
    return None


  ## If the Budget value is a list
  if isinstance(string, list):
    substrings = [s for s in string if '$' in s]

    box_office_list =[]
    for sub in substrings:
      money_string = get_money_string(sub)
      box_office_list.append(word_to_number(money_string))
    return sum(box_office_list)
  
  else:
    string = string.lower()
    ## If there are 2 revenues in a string
    if (string.count('$') > 1) and ('–' not in string): 
      pattern = r'[US]*\$[0-9.]+[0-9\-\$\s\w\W]* million|\$[0-9\,\.]+'
      box_office_list = []
      for sub in re.findall(pattern, string):
        box_office_list.append(word_to_number(sub.replace('$', '')))
      return sum(box_office_list)
    
    else:
      money_string = get_money_string(string)
      return word_to_number(money_string)

In [ ]:
for movie in disney_data:
  try:
    movie['BoxOffice_converted'] = get_box_office(movie.get('Box office', 'N/A'))
  except Exception as e:
    print(e)
    print(movie['Box office'], movie['Title'])


'NoneType' object has no attribute 'group'
US$ 204.8 million Ponyo


In [ ]:
## Alright, we only need to fix the Ponyo movie with the Box office of 204.8 million

for movie in disney_data:
  if movie['Title'] == 'Ponyo':
    movie['BoxOffice_converted'] = '204800000'
  # if movie['Title'] == 'Jagga Jasoos':
  #   movie['BoxOffice_converted'] = None

  print([movie.get('Box office', 'N/A'), movie['BoxOffice_converted']])

['$45.472', 45.472]
['$418 million', 418000000.0]
['$164 million', 164000000.0]
['$76.4–$83.3 million (United States and Canada)', 76400000.0]
['$960,000 (worldwide rentals)', 960000.0]
['>$1.3 million (est. United States/Canada rentals, 1941)', 1300000.0]
['$267.4 million', 267399999.99999997]
['$1.135 million (worldwide rentals)', 1135000.0]
['$799,000', 799000.0]
['$3.355 million (worldwide rentals)', 3355000.0]
['$3.275 million (worldwide rentals)', 3275000.0]
['$65 million', 65000000.0]
['$3.165 million (worldwide rentals)', 3165000.0]
['$2.56 million (worldwide rentals)', 2560000.0]
['$3.7 million (U.S. rental) $575,000 (foreign rental)', 4275000.0]
['$1.625 million (worldwide rentals)', 1625000.0]
['$182 million', 182000000.0]
['$4,100,000 (worldwide rentals)', 4100000.0]
[['$2.4 million (1951, domestic)', '$3.5 million (1974, domestic)'], 5900000.0]
['$2.1 million (US rentals)', 2100000.0]
['$87.4 million (United States and Canada)', 87400000.0]
['$1 million (US)', 1000000.0]
[

#### 3.5. Convert datetime format

In [ ]:
## Get release dates

[[movie.get('Release dates', 'N/A'), movie.get('Release date', 'N/A'), movie['Title']] for movie in disney_data]

In [ ]:
import re
from datetime import datetime

def date_convert(string):
  '''
  Get the datetime string 
  '''

  # pattern1 = r'(?P<month>[A-Z]\w*) (?P<date>\d{1,2}), (?P<year>\d{4})'
  # pattern2 = r'(?P<date>\d{1,2}) (?P<month>[A-Z]\w*) (?P<year>\d{4})'
  
  pattern1 = r'[A-Z]\w* \d{1,2}, \d{4}'
  pattern2 = r'\d{1,2} [A-Z]\w* \d{4}'
  pattern3 = r'\d{4}'
  match1 = re.search(pattern1, string)
  match2 = re.search(pattern2, string)
  match3 = re.search(pattern3, string)

  if match1 != None:
    date = datetime.strptime(match1.group(), '%B %d, %Y').strftime('%b %d, %Y')
    return date

  elif match2 != None:
    date = datetime.strptime(match2.group(), '%d %B %Y').strftime('%b %d, %Y')
    return date

  elif match3 != None:
    date = match3.group()
    return date
  
  # else:
  #   date = None
  #   return date


def get_date_string(string):

  if string in ['N/A', '', 'unknown']:
    return None
  
  if isinstance(string, list):
    for sub in string:
      date = date_convert(sub)
      if date != None:
        return date
      else:
        continue

  else:
    date = date_convert(string)
    return date




  


In [ ]:
for movie in disney_data:
  try:
    if 'Release date' in movie.keys():
      movie['Release_date_converted'] = get_date_string(movie['Release date'])
    elif 'Release dates' in movie.keys():
      movie['Release_date_converted'] = get_date_string(movie['Release dates'])
    else:
      movie['Release_date_converted'] = None
  except Exception as e:
    print(e)
    print(movie['Title'])


In [ ]:
[[movie.get('Release date', 'N/A'), movie.get('Release dates', 'N/A'), movie['Release_date_converted']] for movie in disney_data]